In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np


In [2]:

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

X_train = X_train/255
X_test = X_test/255

y_train = tf.one_hot(y_train, 10)
y_test = tf.one_hot(y_test, 10)

X_train = np.reshape(X_train, (-1, 28, 28, 1))
X_test = np.reshape(X_test, (-1, 28, 28, 1))

In [3]:
class ResidualBlock(tf.keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=units, kernel_size=(3, 3),  padding="same", activation="relu")
        self.conv2 = tf.keras.layers.Conv2D(filters=units, kernel_size=(3, 3),  padding="same", activation="relu")
        self.conv3 = tf.keras.layers.Conv2D(filters=units, kernel_size=(3, 3),  padding="same", activation="relu")
    
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        x = self.conv3(x)
        output = inputs + x
        return output


In [4]:
class ResNet(tf.keras.models.Model):
    def __init__(self):
        super().__init__()
        self.residual_block1 = ResidualBlock(20)
        self.max_pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="valid")
        self.residual_block2 = ResidualBlock(20)
        self.max_pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="valid")
        self.residual_block3 = ResidualBlock(20)
        self.max_pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2), padding="valid")
        self.gap = tf.keras.layers.GlobalAveragePooling2D()
        self.dense = tf.keras.layers.Dense(units=10, activation="softmax")

    def call(self, inputs):
        x = self.residual_block1(inputs)
        x = self.max_pool1(x)
        x = self.residual_block2(x)
        x = self.max_pool2(x)
        x = self.residual_block3(x)
        x = self.max_pool3(x)
        x = self.gap(x)
        output = self.dense(x)
        return output


In [5]:

resnet = ResNet()

dataset_train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(512)

optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.CategoricalCrossentropy()
accuracy_object = tf.keras.metrics.CategoricalAccuracy()


In [6]:

for i in range(25):
    losses = []
    accuracies = []
    for X, y in dataset_train:
        with tf.GradientTape() as tape:
            out = resnet(X)
            loss = loss_object(y, out)
        losses.append(loss)
        accuracies.append(accuracy_object(y, out))
        gradients = tape.gradient(loss, resnet.trainable_variables)
        xyz = optimizer.apply_gradients(zip(gradients, resnet.trainable_variables))
    mean_loss = np.mean(losses)
    mean_accuracy = np.mean(accuracies)
    print("Epoch: " + str(i+1) + " Loss: " + str(mean_loss) + " Accuracy: " + str(mean_accuracy))





If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch: 1 Loss: 1.4143388 Accuracy: 0.2936581
Epoch: 2 Loss: 0.30549046 Accuracy: 0.6444586
Epoch: 3 Loss: 0.19799301 Accuracy: 0.7613505
Epoch: 4 Loss: 0.15542546 Accuracy: 0.8153295
Epoch: 5 Loss: 0.13188623 Accuracy: 0.8468857
Epoch: 6 Loss: 0.11603147 Accuracy: 0.868033
Epoch: 7 Loss: 0.09723568 Accuracy: 0.8834346
Epoch: 8 Loss: 0.08626835 Accuracy: 0.8953596
Epoch: 9 Loss: 0.07657892 Accuracy: 0.9048041
Epoch: 10 Loss: 0.06878967 Accuracy: 0.91254103
Epoch: 11 Loss: 0.06180538 Accuracy: 0.9190402
E

In [7]:
y_pred = resnet.predict(X_test)
accuracy = accuracy_object(y_test, y_pred)
print(accuracy.numpy())


0.9589093
